In [1]:
from openml import tasks

import torch

import sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold

from classes import Dataset, GroupStructure, WeightClipper
from functions import run_eagga_cv

In [2]:
oml_task_diabetes = tasks.get_task(37)

In [3]:
X, y, categorical_indicator, attribute_names = oml_task_diabetes.get_dataset().get_data()

In [4]:
X

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,tested_positive
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,tested_negative
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,tested_positive
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,tested_negative
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,tested_negative
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,tested_negative
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,tested_positive


In [5]:
Xy = X.copy()

In [6]:
gs = GroupStructure(
    {0, 1, 2, 3, 4, 5, 6, 7},
    {0, 1},
    ((2, 5), 1),
    ((4,), 0),
    ((7, 3, 6), 1)
)

tmp = Dataset(
    X=Xy.loc[:, Xy.columns != 'class'],
    y=Xy.loc[:, 'class'],
    class_pos='tested_positive',
    group_structure=gs
)
len(tmp)
tmp[2]

gs.get_unconstrained_features()

{4}

In [7]:
categorical_indicator

[False, False, False, False, False, False, False, False, True]

In [8]:
# outer
data_train_test, data_val = train_test_split(
    Xy,
    train_size=2/3,
    shuffle=True,
    stratify=Xy.loc[:, 'class']
)

# reset indices as StratifiedKFold assumes 0-(n-1) index
data_train_test = data_train_test.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)

# inner
cv_inner = StratifiedKFold(
    n_splits=5,
    shuffle=False  # TODO: set to True
)

'''
in each inner fold, run EAGGA + find an optimal pareto front of configurations for this specific fold
then in the outer fold (holdout), compare each fold-specific optimal pareto front + select optimum
'''
mu = 3  # TODO: set to 100
la = 1  # TODO: set to 10
monotonicity_clipper = WeightClipper(0, None)  # enforce monotonicity by clipping weights to [0, infty) after each epoch (in def train)
run_eagga_cv(mu, la, cv_inner, data_train_test, epochs=10, batch_size=8, weight_clipper=monotonicity_clipper)

running HPO for individual 1/3: 3 total_layers, 4 nodes per hidden layer
fold 1/5
(0.45439469320066334, 0.75, 0.14285714285714285, 0.125)
fold 2/5
(0.5344112769485904, 0.75, 0.14285714285714285, 0.125)
fold 3/5
(0.6622601279317697, 0.75, 0.14285714285714285, 0.125)
fold 4/5
(0.5218855218855218, 0.75, 0.14285714285714285, 0.125)
fold 5/5
(0.5462962962962963, 0.75, 0.14285714285714285, 0.125)
running HPO for individual 2/3: 5 total_layers, 3 nodes per hidden layer
fold 1/5
(0.4950248756218906, 0.75, 0.14285714285714285, 0.125)
fold 2/5
(0.5, 0.75, 0.14285714285714285, 0.125)
fold 3/5
(0.6908315565031983, 0.75, 0.14285714285714285, 0.125)
fold 4/5
(0.4558080808080808, 0.75, 0.14285714285714285, 0.125)
fold 5/5
(0.48274410774410775, 0.75, 0.14285714285714285, 0.125)
running HPO for individual 3/3: 6 total_layers, 6 nodes per hidden layer
fold 1/5
(0.4423714759535655, 0.75, 0.14285714285714285, 0.125)
fold 2/5
(0.5912106135986732, 0.75, 0.14285714285714285, 0.125)
fold 3/5
(0.69808102345415